In [11]:
!pip install demoji

In [12]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import hashlib
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FunctionTransformer, Pipeline
import demoji
from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer('english')
stopword = stopwords.words('english')
demoji.download_codes()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-12-a9cb014de6a9>:18: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [13]:
train_dataframe = pd.read_csv('train.csv')
test_dataframe = pd.read_csv('test.csv')

In [25]:
class feature_extractor(BaseEstimator, TransformerMixin):
    def __init__(self, features: list[str]):
        super().__init__()
        self.features = features

    def transform(self, X):
        columns_to_drop = set(X) - set(self.features)

        X_transformed = X.drop(columns_to_drop, axis=1)

        return X_transformed


# class clear_columns(BaseEstimator, TransformerMixin):
#     def __init__(self, features):
#         super().__init__()
#         self.features = features

#     def transform(self, data):
#         pattern = re.compile(r'[@|#][^\s]+')
#         df_list = []
#         for row in data.itertuples(index=False):
#             df_dict = {}
#             for feature in self.features:
#                 column = data.columns.get_loc(feature)
#                 value = re.sub(r'[^a-zA-Z\s]', '', row[column])
#                 value = re.sub(r'\b\w{1,2}\b', '', value)
#                 value = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', '', value)
#                 value = demoji.replace_with_desc(value)
#                 value = re.sub(r'[@|#][^\s]+', '', value)
#                 value = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', value)
#                 value_list = value.split()
#                 value = ' '.join([stemmer.stem(i) for i in value_list if i not in stopword])

#             df_dict['Cleaned_data'] = value
#             df_list.append(df_dict)

#         return pd.DataFrame(df_list)


class clear_columns(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, data):
        demoji.download_codes()
        for feature in self.features:
            if feature in data.columns:
                data[feature] = data[feature].apply(self.clean_text)
        return data

    def clean_text(self, text):
        if pd.isnull(text):
            return text
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\b\w{1,2}\b', '', text)
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', '', text)
        text = demoji.replace_with_desc(text)
        text = re.sub(r'[@|#][^\s]+', '', text)
        text = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', text)
        value_list = text.split()
        value = ' '.join([stemmer.stem(i) for i in value_list if i not in stopword])
        return value


# class merge_columns(BaseEstimator, TransformerMixin):
#     def __init__(self, main_column, features):
#         super().__init__()
#         self.main_column = main_column
#         self.features = features

#     def transform(self, data):

In [26]:
def merging_details(data):
        df_list = []

        for row in data.itertuples():
            df_dict = {}
            # Processing Keyword and location
            keyword = re.sub(r'[^a-zA-Z\s]','', str(row[2]))
            location = re.sub(r'[^a-zA-Z\s]','', str(row[3]))
            keyword = re.sub(r'\b\w{1,2}\b', '', keyword)
            location = re.sub(r'\b\w{1,2}\b', '', location)
            text = str(row[4])

            if keyword == 'nan':
                if location == 'nan':
                    prs_data = text
                else:
                    prs_data = location + ' ' + text
            else:
                if location == 'nan':
                    prs_data = keyword + ' ' + text
                else:
                    prs_data = keyword + ' ' + location + ' ' + text

            prs_data = re.sub(' +', ' ', prs_data)
            prs_data = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', '', prs_data)
            prs_data =  demoji.replace_with_desc(prs_data)

            df_dict['text'] = prs_data.lower()

            df_list.append(df_dict)

        return pd.DataFrame(df_list)


In [27]:
from sklearn.pipeline import Pipeline

transformer = clear_columns(features=['text'])

pipeline = Pipeline(
    [
        ('clear_columns', transformer),
    ]
)

train_dataframe_cleaned = pipeline.fit_transform(train_dataframe)

train_dataframe_cleaned.head()


<ipython-input-25-00b9aa75cce7>:49: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()  # Ensure that demoji is ready


,id,keyword,location,text,target
0,1,NaN,NaN,our deed reason earthquak may allah forgiv,1
1,4,NaN,NaN,forest fire near rong sask canada,1
2,5,NaN,NaN,all resid ask shelter place notifi offic evacu...,1
3,6,NaN,NaN,peopl receiv wildfir evacu order california,1
4,7,NaN,NaN,just got sent photo rubi alaska smoke wildfir ...,1


In [23]:
transformer = (
    f"transformer",
    clear_columns(['text'])
)

pipeline = Pipeline(
    [
        feature_extractor,
    ]
)

train_dataframe = pipeline.transform(train_dataframe)

train_dataframe.head()

AttributeError: This 'Pipeline' has no attribute 'transform'

In [1]:
def lower_text(text: str) -> str:
    return text.lower()

def remove_numbers(text: str) -> str:
    """
    Substitute all punctuations with space in case of
    "there is5dogs".

    If subs with '' -> "there isdogs"
    With ' ' -> there is dogs
    """
    text_nonum = re.sub(r'\d+', ' ', text)
    return text_nonum

def remove_punctuation(text: str) -> str:
    """
    Substitute all punctiations with space in case of
    "hello!nice to meet you"

    If subs with '' -> "hellonice to meet you"
    With ' ' -> "hello nice to meet you"
    """
    # Write your code here
    text_nopunct = re.sub(r'[^\w\s]', ' ', text)
    return text_nopunct

def remove_multiple_spaces(text: str) -> str:
    text_no_doublespace = re.sub(r'\s+', ' ', text).strip()
    return text_no_doublespace

def tokenize_text(text: str) -> list[str]:
    return word_tokenize(text)

def remove_stop_words(tokenized_text: list[str]) -> list[str]:
    stop_words = set(stopwords.words('english'))
    return [word for word in tokenized_text if word.lower() not in stop_words]

def stem_words(tokenized_text: list[str]) -> list[str]:
    return [PorterStemmer().stem(word) for word in tokenized_text]

In [2]:
def preprocessing_text(text):
    _lowered = lower_text(text)
    _without_numbers = remove_numbers(_lowered)
    _without_punct = remove_punctuation(_without_numbers)
    _single_spaced = remove_multiple_spaces(_without_punct)
    _tokenized = tokenize_text(_single_spaced)
    _without_sw = remove_stop_words(_tokenized)
    _stemmed = stem_words(_without_sw)

    return _stemmed

In [4]:
def drop_id(df):
    """
    Drop id column
    """
    df.drop('Title', axis=1, inplace=True)
    return df

def clean_text_inplace(df):
    df['Text'] = df['Text'].apply(preprocessing_text)
    return df

In [ ]:
train_copy = train_dataframe.head().copy()

clean_text_inplace(merging_details(train_copy))

